# <center>Stub Functions and Automatic Documentation</center>

<img src="https://pdoc3.github.io/pdoc/logo.png" width=30% alt="pdoc3 logo">


The goal of this project is to solidify and document the major structure of your project code.  We will do this by adding stub functions (where needed) and using the [pdoc3 automatic documentation generator](https://pdoc3.github.io/pdoc/).  An automatic documentation generator takes comments formated in your Python source code and uses them to generate your project documentation.  This way you only need to change one set of files.  



# Stubbing out your code

A stub function in software development is like an outline used to stand in as a substitute before the details of the function can be completed.  Good stub functions are like a plan for what you want your code to do. It takes the expected inputs and returns outputs in the expected format.  

For example, if you're planning to build an alarm for your freezer to let you know if the temperature gets too warm, you might write the following stubs.

In [1]:
def ThermometerRead(myThermometer):
    # This is where I will put the code to read from my thermometer.
    return 4

freezer = None
temperature = ThermometerRead(freezer)
if temperature > 10:
    # Reminder to myself that the temperature should be in celsius. 
    print ("Warning: The freezer might be broken")


In [1]:
def ReadCSV(csv):
    import pandas as pd
    import numpy as np 
    from sklearn import preprocessing
    import matplotlib.pyplot as plt 
    %matplotlib inline
    import seaborn as sns #can i just say i REALLY hate how people import seaborn as sns. just import it as sbn COME ON
    from scipy.stats import spearmanr, pearsonr
    sns.set(style="white") #white background style for seaborn plots
    sns.set(style="whitegrid", color_codes=True)
    """
    Imports all future libraries
    Reads in CSV file according to Pandas library
    spits out file
    
    """
    file = pd.read_csv(csv)
    return file


In [4]:
from scipy.stats import spearmanr, pearsonr

In [5]:
def calculate_pvalues(df,method = spearmanr):
    """
    Assumes df with only numeric entries clean of null entries. 
    Calls df in to perform spearmanr method from scipy.stats.
    Transposes the columns to provide p values for each column.
    """
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(method(df[r], df[c])[1], 4)
    return pvalues

In [6]:
def correlation_matrix(df,
                       method = "pearson",
                       annot_bool = False,
                       annot_size = 20
                      ):
    # Compute the correlation matrix
    corr = df.corr(method = method)
    """
    Setting up a correlation matrix using the pearson method from scipy.stats.
    """

    if annot_bool:
        annot = corr.copy()
        
        if method == "pearson":
            sig_meth = pearsonr
        else:
            sig_meth = spearmanr
            
        pval = calculate_pvalues(df, sig_meth) 
        # create three masks
        r0 = corr.applymap(lambda x: '{:.2f}'.format(x))
        r1 = corr.applymap(lambda x: '{:.2f}*'.format(x))
        r2 = corr.applymap(lambda x: '{:.2f}**'.format(x))
        r3 = corr.applymap(lambda x: '{:.2f}***'.format(x))
  
        # apply them where appropriate --this could be a single liner
        annot = annot.where(pval>0.1,r0)
        annot = annot.where(pval<=0.1,r1)
        annot = annot.where(pval<=0.05,r2)
        annot = annot.mask(pval<=0.01,r3)
    else:
        annot = False

    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    
    """
    Map creation; more vivid color = more correlated, one of these features should be removed from consideration.
    """

    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 11))

    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)

    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5, 'label': 'Pearson'},
                annot = annot,
                fmt = "",
                annot_kws={"size": annot_size},
                vmin = -1,
                vmax = 1,
               )

In [ ]:
def findfeatures(file):
    #take file: translate into which columns are of interest (finished dataset)
    #make new table with trimmed columns?
    #define lines for each feature into separate tables
    #return lists, lists, lists
    columns = list(file.columns)
    return columns

As you can see, the code doesn't actually do what I want yet, but it will run, and I have the overall structure set up.  So, once I have my roadmap set up, I can go back and fill in the pieces. 

For this project we want to stub out the main structure of your programming interface.  How the functions are stubbed out will depend highly on your project.  The idea is to plan how you want the code functions and libraries to interact before writing in all of the details.  Good stubbing will make it much easier to incrementally develop and test your software.  

&#9989; **<font color=red>DO THIS:</font>**  Stub out the main functions and classes for your code.  Commit your changes to your git repository.

# Docstrings

The Python programming language has some standard ways that functions and classes should be documented.  Strings surrounded by triple quotes (""") located at the beginning of documents, functions and classes are used to document those items.  Consider the following example function:

In [2]:
def testfunction(a, b):
    """
    This is a test function documentation
    You can put your stuff here
    """
    print(f"running testfunction {a} {b}")
    return 0

Docstrings are the primary way that Python uses to print ```help``` messages. For example:

In [3]:
help(testfunction)

Help on function testfunction in module __main__:

testfunction(a, b)
    This is a test function documentation
    You can put your stuff here



&#9989; **<font color=red>DO THIS:</font>**  Read the [PEP257](https://www.python.org/dev/peps/pep-0257/) standard and write docstrings for all of your files, functions and classes. Commit your changes to your git repository. (another good resource is  [this post on DocStrings](https://www.datacamp.com/community/tutorials/docstrings-python))

&#9989; **<font color=red>DO THIS:</font>**  Install and use [```pydocstyle```](http://www.pydocstyle.org/) to check and ensure the documentation meets the PEP257 standard as best it can. Commit any additional changes to your git repository. See instructions in next section for example installation code. 

    conda install pydocstyle
    pydocstyle modulename

# Automatic Documentation using [```pdoc3```](https://pdoc3.github.io/pdoc/)

The ```help``` function is not the only Python module that can take advantage of doc strings.  There are a class of programs called "auto documentors" that can read though code, organize the doc strings and output documentation for your software in a variety of formats (ex. pdf and html).  Probably the most famous of these is [Sphinx](http://www.sphinx-doc.org/).  However, ```Sphinx``` is really intended for large software projects and is a little overkill for our class project.  For this project we plan to use ```pdoc``` which is a much easier to use program and fully compatible with ```Sphinx```. Meaning we can use ```pdoc``` to organize our documentation and easily upgrade to ```Sphinx``` if/when that is deemed necessary.  

**Notes:** 
- Although it is possible to get pdoc3 running on Jupyterhub the steps are much easier if you run from your local Anaconda install. We highly recommend using the local install.
- There are many different programs for doing auto-documentation. If you have more experience with some other option, feel free to use something else. See a list of examples [here](https://wiki.python.org/moin/DocumentationTools) and some discussion of comparisons [here](https://medium.com/@peterkong/comparison-of-python-documentation-generators-660203ca3804). 

---
<a name="Quick_pdoc3_practice"></a>

## pdoc3 install and (optional) practice 

Let's just try to get this working on some tester code. 


&#9989; **<font color=red>DO THIS:</font>** Install pdoc on your machine.  If you are using jupyterhub, use the following command in bash:


```bash
    pip install --user pdoc3
```

Or, if you prefer, you can use conda environments and install using the following:


```bash
    pip install pdoc3
```

**Warning:** Make sure you install ```pdoc3``` for use in your project.  Older versions of ```pdoc``` are much harder to use. 

Once pdoc3 is installed they may need to add "~/.local/bin/" to your terminal path. In jupyterhub I recommend adding the following to your .bashrc file and restarting the terminal:

```bash
    PATH=$PATH:~/.local/bin/
```


To test out the setup, try pdoc on a single python file. I have included a file called ```graphweb.py``` in the class repo for you to test this.
```bash
   pdoc3 --force --html --output-dir ./docs graphweb.py
```


Open up the docs folder using your file manager and click on one of the html files to see the output displayed in your internet browser.

Here is the main command to get pdoc working:

    pdoc --force --html --output-dir ./docs modulename

&#9989; **<font color=red>DO THIS:</font>** Get ```pdoc3``` working with your project code and generate html files in a docs folder using the above.  Commit the html files to the folder and push all of your changes to your git central repository. **_HINT_** Use the ```make docs``` command to run pdoc3 on your repository.

# Publishing on Github.io (Optional)
 
Note, using [github](https://github.com/) is **_NOT_** required for this project.  However, github has a interesting option to make a website out of html files committed to a repository.  These webpages appear for free on github.io.  Make your own project website by doing the following steps:

1. Generate your html files in your doc folder.
2. Commit and push all of the generated html files to your git repository. 
3. Go to http://github.com project folder and select settings.
4. Under the "GitHub Pages" section in settings select "main branch /docs folder." 
5. Wait a few minutes and go to `username.github.io/projectname` and view your pages. 

---
### Getting credit for your assignment

Now, you just need to commit and push this report to your project git repository. Your instructor will download your repository and check your automatic documentation using the ```make docs``` command so make sure that is working. 

-----
### Congratulations, you are done!